In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import matplotlib.pyplot as plt
import polars as pl
from datetime import datetime, time

from ts.supplier import TradeTick
from ts.supplier import TickSupplier

from ts.supplier import Bar
from ts.supplier import BarSupplier
from ts.supplier import BarAggregation
from ts.supplier import BarFeatureSupplier, BarFeature
from ts.supplier import RollingFeatureSupplier, SupplierType
from ts.supplier import Function, BaseSupplier
from ts.supplier import MultiplexSupplier

filepath = "/Users/jacobgora/projects/data/continuous_futures/CME-HO.parquet"
tick_supplier = TickSupplier(instrument="CME-HO")
tick_supplier.from_parquet(filepath)

In [4]:
tick_supplier.data.columns

['timestamp', 'price', 'quantity', 'side', '__index_level_0__']

In [5]:
tick_supplier.data.head(5)

timestamp,price,quantity,side,__index_level_0__
"datetime[ns, US/Eastern]",f64,i64,i64,"datetime[ns, US/Eastern]"
2019-12-02 19:00:00 EST,18853.0,1,1,2019-12-02 19:00:00 EST
2019-12-02 19:01:01 EST,18854.0,1,1,2019-12-02 19:01:01 EST
2019-12-02 19:11:00 EST,18857.0,1,0,2019-12-02 19:11:00 EST
2019-12-02 19:12:00 EST,18857.0,1,0,2019-12-02 19:12:00 EST
2019-12-02 19:17:11 EST,18856.0,1,0,2019-12-02 19:17:11 EST


In [6]:
tick_supplier.data

timestamp,price,quantity,side,__index_level_0__
"datetime[ns, US/Eastern]",f64,i64,i64,"datetime[ns, US/Eastern]"
2019-12-02 19:00:00 EST,18853.0,1,1,2019-12-02 19:00:00 EST
2019-12-02 19:01:01 EST,18854.0,1,1,2019-12-02 19:01:01 EST
2019-12-02 19:11:00 EST,18857.0,1,0,2019-12-02 19:11:00 EST
2019-12-02 19:12:00 EST,18857.0,1,0,2019-12-02 19:12:00 EST
2019-12-02 19:17:11 EST,18856.0,1,0,2019-12-02 19:17:11 EST
2019-12-02 19:32:21 EST,18859.0,1,0,2019-12-02 19:32:21 EST
2019-12-02 19:33:34 EST,18864.0,1,1,2019-12-02 19:33:34 EST
2019-12-02 20:00:00 EST,18850.0,1,0,2019-12-02 20:00:00 EST
2019-12-02 20:00:00.001 EST,18849.0,1,0,2019-12-02 20:00:00.001 EST


In [7]:
suppliers = [
    BarFeatureSupplier(
        supplier=BarSupplier(
            supplier=tick_supplier,
            bar_aggregation=BarAggregation.TIME_MINUTES,
            size=5
        )
    ),
    BarFeatureSupplier(
        supplier=BarSupplier(
            supplier=tick_supplier,
            bar_aggregation=BarAggregation.TIME_MINUTES,
            size=1
        )
    )
]

multiplex_supplier = MultiplexSupplier(suppliers=suppliers)

In [8]:
rolling_feat_supplier = RollingFeatureSupplier(
    supplier=multiplex_supplier,
    functions=[Function.Z_SCORE],
    type_attributes=[
        # OFI / Tape
        BarFeature.OFI_NORMALIZED,
        BarFeature.OFI,

        BarFeature.ASK_SIZE,
        BarFeature.BID_SIZE,

        BarFeature.ASK_SIZE_TIMEDELTA,
        BarFeature.BID_SIZE_TIMEDELTA,

        BarFeature.RETURN_TIMEDELTA,

        BarFeature.RETURN_BID_SIZE,
        BarFeature.RETURN_ASK_SIZE,

        # Volume
        BarFeature.CUMULATIVE_VOLUME_DELTA,
        BarFeature.VOLUME_DELTA,
        BarFeature.VOLUME,

        # Volatility
        BarFeature.POS_REALIZED_VARIANCE,
        BarFeature.NEG_REALIZED_VARIANCE,

        #
        BarFeature.INTERNAL_BAR_STRENGTH
    ]
)

In [9]:
rolling_feat_supplier.data[
    [
        e for e in rolling_feat_supplier.data.columns
        if "rolling_features" in e
    ]
].tail()

rolling_features-bar_features-bar-CME-HO-time_minutes_agg-1-ofi_normalized-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-5-ofi_normalized-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-1-ofi-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-5-ofi-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-1-ask_size_timedelta-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-5-ask_size_timedelta-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-1-bid_size_timedelta-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-5-bid_size_timedelta-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-1-return_timedelta-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-5-return_timedelta-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-1-volume_delta-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-5-volume_delta-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-1-volume-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-5-volume-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-1-pos_realized_variance-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-5-pos_realized_variance-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-1-neg_realized_variance-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-5-neg_realized_variance-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-1-internal_bar_strength-z_score-10,rolling_features-bar_features-bar-CME-HO-time_minutes_agg-5-internal_bar_strength-z_score-10
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.659223,0.680917,1.037562,17.078371,-0.433195,-0.847683,NaN,NaN,-0.316218,-0.382113,-1.037562,-17.078371,0.236711,17.938541,-1.161894,-0.948683,-0.755778,-0.929998,-0.621059,-1.873314
0.303857,0.659366,1.523175,17.01613,-0.433195,-0.796251,NaN,NaN,-0.316227,-0.574197,-1.523175,-17.01613,1.147112,17.927575,-1.430313,-0.948683,-0.621059,-0.929998,0.225403,-1.720809
-1.453718,0.318571,-1.444478,16.690439,-0.377033,-0.699057,NaN,NaN,-0.316194,-0.625675,1.444478,-16.690439,0.23825,17.902931,-1.161894,-0.948683,-0.755773,-0.929998,-0.774597,-1.525668
0.149386,-0.326637,8.409404,16.148576,-0.485004,-1.59215,NaN,NaN,-0.316204,-0.682677,-8.409404,-16.148576,12.198141,17.862311,-1.143063,-0.948683,-0.774597,-0.929998,-0.312207,-1.372188
-0.473929,-1.522993,4.253971,13.932494,-0.623085,-1.611916,NaN,NaN,-0.316201,-0.729918,-4.253971,-13.932494,10.06066,25.767978,-1.161895,-1.143209,-0.755752,-0.774597,-0.722463,-1.788854


In [10]:
rolling_feat_supplier.data.columns

['bar-CME-HO-time_minutes_agg-1-open',
 'bar-CME-HO-time_minutes_agg-1-low',
 'bar-CME-HO-time_minutes_agg-1-high',
 'bar-CME-HO-time_minutes_agg-1-close',
 'bar-CME-HO-time_minutes_agg-1-volume',
 'bar-CME-HO-time_minutes_agg-1-timestamp',
 'bar-CME-HO-time_minutes_agg-1-timedelta',
 'bar-CME-HO-time_minutes_agg-1-ask_size',
 'bar-CME-HO-time_minutes_agg-1-bid_size',
 'bar-CME-HO-time_minutes_agg-1-return',
 'bar-CME-HO-time_minutes_agg-1-log_return',
 'bar_features-bar-CME-HO-time_minutes_agg-1-return_timedelta',
 'bar_features-bar-CME-HO-time_minutes_agg-1-bid_size_timedelta',
 'bar_features-bar-CME-HO-time_minutes_agg-1-ask_size_timedelta',
 'bar_features-bar-CME-HO-time_minutes_agg-1-volume_delta',
 'bar_features-bar-CME-HO-time_minutes_agg-1-volume',
 'bar_features-bar-CME-HO-time_minutes_agg-1-ofi',
 'bar_features-bar-CME-HO-time_minutes_agg-1-ofi_normalized',
 'bar_features-bar-CME-HO-time_minutes_agg-1-open_high',
 'bar_features-bar-CME-HO-time_minutes_agg-1-open_low',
 'bar_f